In [5]:
import undetected_chromedriver as uc
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import pandas as pd
import time
from bs4 import BeautifulSoup
from selenium.common.exceptions import NoSuchElementException, TimeoutException, WebDriverException

driver = uc.Chrome()
driver.maximize_window()

base_url = 'https://shop.rebag.com/collections/all-bags?_=pf&pf_st_availability_hidden=true&pf_t_categories=bc-filter-Bags&g_acctid=218-424-2787&g_adgroupid=&g_adid=&g_adtype=none&g_campaign=PMax%3AGoogle_ZombieSKU&g_campaignid=18397495333&g_keyword=&g_keywordid=&g_network=x&utm_campaign=18397495333.&utm_content=%7Bcreative%5D&utm_medium=cpc&utm_source=google&gad_source=1&gclid=CjwKCAjwlbu2BhA3EiwA3yXyu2HsfbNjOK118WDSLzut0JE82kweVRfH6IU7A8BktLGs3iBSaWBVNhoCsZEQAvD_BwE'
page = 1
data = []

while True:
    page_url = f'{base_url}&page={page}'
    print(f"Scraping page: {page}")

    try:
        driver.get(page_url)
        WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.CLASS_NAME, 'plp__product')))
        soup = BeautifulSoup(driver.page_source, 'html.parser')
        products = soup.find_all('div', class_='plp__product')

        if not products:
            print("No products found on this page, stopping scraping.")
            break

        for product in products:
            try:
                product_url_tag = product.find('a', class_='plp__card')
                if product_url_tag:
                    product_url = product_url_tag.get('href')
                    if not product_url.startswith('http'):
                        product_url = 'https://shop.rebag.com' + product_url
                    print(f"Product URL: {product_url}")

                    driver.get(product_url)
                    WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.CLASS_NAME, 'pdp__description')))
                    soup_product = BeautifulSoup(driver.page_source, 'html.parser')
                    img_tag = soup_product.find('div', class_='pdp__desktop-image')
                    img_src = 'Image src attribute not found'
                    if img_tag:
                        img = img_tag.find('img')
                        if img and img.has_attr('src'):
                            img_src = img['src']
                            if img_src.startswith('//'):
                                img_src = 'https:' + img_src

                    designer_tag = soup_product.find('a', class_='pdp__designer')
                    designer = designer_tag.text.strip() if designer_tag else 'Designer not found'

                    title_tag = soup_product.find('h1', class_='pdp__header')
                    title = 'Title not found'
                    if title_tag:
                        title_a_tag = title_tag.find('a', class_='pdp__title')
                        if title_a_tag:
                            title = title_a_tag.text.strip()

                    price_section = soup_product.find('div', class_='pdp__price')
                    standard_price = 'Standard Price not found'
                    if price_section:
                        standard_price_tag = price_section.find('span', class_='rewards-plus-pdp__product-price-value')
                        if standard_price_tag:
                            standard_price = standard_price_tag.text.strip()

                    size_and_fit_div = soup_product.find('div', class_='pdp__section--size-and-fit')
                    size_and_fit = 'Size and Fit not found'
                    if size_and_fit_div:
                        size_and_fit_tag = size_and_fit_div.find('div', class_='pdp__group-item')
                        if size_and_fit_tag:
                            size_and_fit = size_and_fit_tag.text.strip()

                    overview_div = soup_product.find('div', class_='pdp__section--overview')
                    overview_details = {}
                    if overview_div:
                        groups = overview_div.find_all('div', class_='pdp__section-group')
                        for group in groups:
                            title_tag = group.find('div', class_='pdp__group-title')
                            item_tag = group.find('div', class_='pdp__group-item')
                            if title_tag and item_tag:
                                section_title = title_tag.text.strip()
                                items = item_tag.find_all('div', class_='pdp__group-item--li')
                                if items:
                                    item_list = [item.text.strip() for item in items]
                                    overview_details[section_title] = ', '.join(item_list)
                                else:
                                    overview_details[section_title] = item_tag.text.strip()

                    additional_details_div = soup_product.find('div', class_='pdp__section--no-title')
                    additional_details = {}
                    if additional_details_div:
                        groups = additional_details_div.find_all('div', class_='pdp__section-group')
                        for group in groups:
                            title_tag = group.find('div', class_='pdp__group-title')
                            item_tag = group.find('div', class_='pdp__group-item')
                            if title_tag and item_tag:
                                section_title = title_tag.text.strip()
                                additional_details[section_title] = item_tag.get_text(strip=True)

                    product_data = {
                        'Product URL': product_url,
                        'Brand': designer,
                        'Title': title,
                        'Standard Price': standard_price,
                        'Image URL': img_src,
                        'Size and Fit': size_and_fit,
                    }
                    if overview_details:
                        product_data.update(overview_details)
                    if additional_details:
                        product_data.update(additional_details)

                    data.append(product_data)

                    print(f"Brand: {designer}")
                    print(f"Title: {title}")
                    print(f"Standard Price: {standard_price}")
                    print(f"Image URL: {img_src}")
                    print(f"Size and Fit: {size_and_fit}")
                    for key, value in overview_details.items():
                        print(f"{key}: {value}")
                    for key, value in additional_details.items():
                        print(f"{key}: {value}")                        
                    print("_____________________________________________________________________________________________")

            except:
                continue

        try:
            driver.get(page_url)
            time.sleep(2)
            next_button = driver.find_element(By.XPATH, '//ul[@class="rbg-pagination__ul"]//li[@class="next"]')
            if 'disabled' in next_button.get_attribute('class'):
                print("No more pages available, stopping.")
                break
            else:
                next_button.click()
                time.sleep(2)
                page += 1
        except:
            break
    except:
        break
    


    df = pd.DataFrame(data)
    print(df)
    df.to_csv('rebag_products.csv', index=False)
    print("Data saved to rebag_products.csv")


Scraping page: 1
Product URL: https://shop.rebag.com/products/handbags-bottega-veneta-loop-crossbody-bag-intrecciato-nappa-mini2868271?groupBy=productClicked
Brand: Bottega Veneta
Title: Loop Crossbody Bag Intrecciato Nappa Mini
Standard Price: $1,090
Image URL: https://shop.rebag.com/cdn/shop/files/286827-1_20Bottega_20Veneta_20Loop_20Crossbody_20Bag_20Intrecciato_20Nappa_20Mini_2D_0002_508x508.jpg?v=1721669901
Size and Fit: 7"W x 4"H x 2.5"D
Exterior Color: Black
Interior Color: Black
Exterior Material: Leather
Interior Material: Suede
Hardware Color: Gold
Accessories: Dust bag
Store Location: Online Only
Estimated Retail: $1,950
CLAIR CODE: HB.BV.NDNM.INN.MI
Item No: 286827/1
Brand Code: NFC Only
_____________________________________________________________________________________________
Product URL: https://shop.rebag.com/products/handbags-gucci-peggy-tote-gg-canvas-large28952764?groupBy=productClicked
Brand: Gucci
Title: Peggy Tote GG Canvas Large
Standard Price: $655
Image URL: 

Brand: Prada
Title: Cuir Double Tote Saffiano Leather Large
Standard Price: $1,460
Image URL: https://shop.rebag.com/cdn/shop/files/296198-1_20Prada_20Cuir_20Double_20Tote_20Saffiano_20Leather_20Large_2D_0002_508x508.jpg?v=1725297379
Size and Fit: 14"W x 11"H x 7"D
Exterior Color: Black
Interior Color: Black, Pink
Exterior Material: Leather
Interior Material: Leather
Hardware Color: Gold
Accessories: Luggage Tag, Poignet, Detachable Strap
Store Location: Online Only
Estimated Retail: $2,990
CLAIR CODE: HB.PR.CUDT.SAL.LG
Item No: 296198/1
Brand Code: 25
_____________________________________________________________________________________________
Product URL: https://shop.rebag.com/products/handbags-chanel-vintage-chain-tote-quilted-lambskin-mini2920191?groupBy=productClicked
Brand: Chanel
Title: Vintage Chain Tote Quilted Lambskin Mini
Standard Price: $1,650
Image URL: https://shop.rebag.com/cdn/shop/files/292019-1_20Chanel_20Petite_20Timeless_20Tote_20Quilted_20Calfskin_2D_0002_508x508

Brand: Chanel
Title: Classic Double Flap Bag Quilted Lambskin Jumbo
Standard Price: $7,530
Image URL: https://shop.rebag.com/cdn/shop/files/288663-18_20Chanel_20Classic_20Double_20Flap_20Bag_20Quilted_20Lambskin_20Jumbo_2D_0002_508x508.jpg?v=1725297708
Size and Fit: 11.5"W x 8"H x 3.5"D
Exterior Color: Black
Interior Color: Red
Exterior Material: Leather
Interior Material: Leather
Hardware Color: Gold
Accessories: Authenticity card
Store Location: Online Only
Estimated Retail: $11,700
CLAIR CODE: HB.CH.CDFL.QULA.JB
Item No: 288663/18
Brand Code: 19527790
_____________________________________________________________________________________________
Product URL: https://shop.rebag.com/products/handbags-saint-laurent-sac-de-jour-bag-leather-large2753051?groupBy=productClicked
Brand: Saint Laurent
Title: Sac de Jour Bag Leather Large
Standard Price: $950
Image URL: https://shop.rebag.com/cdn/shop/files/275305-1_20Saint_20Laurent_20Sac_20de_20Jour_20Bag_20Leather_20Large_2D_0002_508x508.jpg?

Brand: Louis Vuitton
Title: Papillon Pochette Monogram Canvas Mini
Standard Price: $575
Image URL: https://shop.rebag.com/cdn/shop/files/275836-1_20Louis_20Vuitton_20Papillon_20Pochette_20Monogram_20Canvas_20Mini_2D_0002_508x508.jpg?v=1716128258
Size and Fit: 6"W x 3"H x 3"D
Exterior Color: Brown
Interior Color: Brown
Exterior Material: Canvas Coated
Interior Material: Leather
Hardware Color: Gold
Accessories: Dust bag
Store Location: Online Only
CLAIR CODE: HB.LV.PAPC.MNCA.MI
Item No: 275836/1
_____________________________________________________________________________________________
Product URL: https://shop.rebag.com/products/handbags-prada-re-edition-2005-shoulder-bag-tessuto-small2634672?groupBy=productClicked
Brand: Prada
Title: Re-Edition 2005 Shoulder Bag Tessuto Small
Standard Price: $1,100
Image URL: https://shop.rebag.com/cdn/shop/files/263467-2_20Prada_20Re-Edition_202005_20Shoulder_20Bag_20Tessuto_20Small_2D_0002_508x508.jpg?v=1711392405
Size and Fit: 9"W x 5"H x 3"D
Ext

Brand: Louis Vuitton
Title: Navona Pochette Accessoires Damier
Standard Price: $590
Image URL: https://shop.rebag.com/cdn/shop/files/273308-50_20Louis_20Vuitton_20Navona_20Pochette_20Accessoires_20Damier_2D_0002_508x508.jpg?v=1718039772
Size and Fit: 8.5"W x 5"H x 1"D
Exterior Color: Brown
Interior Color: Red
Exterior Material: Canvas Coated
Interior Material: Fabric
Hardware Color: Gold
Accessories: No Accessories, Detachable Strap
Store Location: Online Only
CLAIR CODE: HB.LV.NVPA.DR
Item No: 273308/50
Brand Code: Illegible Code
_____________________________________________________________________________________________
Product URL: https://shop.rebag.com/products/handbags-chanel-vintage-classic-double-flap-bag-quilted-lambskin-medium2733107?groupBy=productClicked
Brand: Chanel
Title: Vintage Classic Double Flap Bag Quilted Lambskin Medium
Standard Price: $6,190
Image URL: https://shop.rebag.com/cdn/shop/files/273310-7_20Chanel_20Vintage_20Classic_20Double_20Flap_20Bag_20Quilted_20L

...

Brand: Louis Vuitton
Title: Neverfull NM Tote Damier MM
Standard Price: $1,810
Image URL: https://shop.rebag.com/cdn/shop/files/274149-15_20Louis_20Vuitton_20Neverfull_20NM_20Tote_20Damier_20MM_2D_0002_508x508.jpg?v=1719249044
Size and Fit: 13"W x 11"H x 7"D
Exterior Color: Brown
Interior Color: Red
Exterior Material: Canvas Coated
Interior Material: Fabric
Hardware Color: Gold
Accessories: Pochette, Dust bag
Store Location: Online Only
CLAIR CODE: HB.LV.NNM.DR.MM
Item No: 274149/15
Brand Code: FL0159 FL0139
_____________________________________________________________________________________________
Product URL: https://shop.rebag.com/products/handbags-prada-chain-flap-crossbody-bag-nappa-gaufre-small28385741?groupBy=productClicked
Brand: Prada
Title: Chain Flap Crossbody Bag Nappa Gaufre Small
Standard Price: $895
Image URL: https://shop.rebag.com/cdn/shop/files/283857-41_20Prada_20Chain_20Flap_20Crossbody_20Bag_20Nappa_20Gaufre_20Small_2D_0002_508x508.jpg?v=1721669379
Size and Fit: 

Brand: Prada
Title: Re-Edition 2000 Hobo Tessuto Mini
Standard Price: $900
Image URL: https://shop.rebag.com/cdn/shop/files/277109-1_20Prada_20Re-Edition_202000_20Hobo_20Tessuto_20Mini_2D_0002_508x508.jpg?v=1716129350
Size and Fit: 9"W x 5"H x 2.5"D
Exterior Color: Black
Interior Color: Black
Exterior Material: Nylon
Interior Material: Fabric
Hardware Color: Silver
Accessories: Clochette
Store Location: Online Only
Estimated Retail: $1,250
CLAIR CODE: HB.PR.REHO.TES.MI
Item No: 277109/1
Brand Code: 255
_____________________________________________________________________________________________
                                           Product URL           Brand  \
0    https://shop.rebag.com/products/handbags-botte...  Bottega Veneta   
1    https://shop.rebag.com/products/handbags-gucci...           Gucci   
2    https://shop.rebag.com/products/handbags-given...        Givenchy   
3    https://shop.rebag.com/products/handbags-botte...  Bottega Veneta   
4    https://shop.rebag.com/

Brand: Celine
Title: Belt Bag Textured Leather Medium
Standard Price: $1,350
Image URL: https://shop.rebag.com/cdn/shop/files/275855-1_20Celine_20Belt_20Bag_20Textured_20Leather_20Medium_2D_0002_508x508.jpg?v=1716129201
Size and Fit: 12"W x 10"H x 8"D
Exterior Color: Gray
Interior Color: Gray
Exterior Material: Leather
Interior Material: Suede
Hardware Color: Gold
Accessories: No Accessories
Store Location: Online Only
CLAIR CODE: HB.CE.BLTB.TXL.ME
Item No: 275855/1
Brand Code: S-GA-4107
_____________________________________________________________________________________________
Product URL: https://shop.rebag.com/products/handbags-louis-vuitton-melie-handbag-monogram-canvas-280868171?groupBy=productClicked
Brand: Louis Vuitton
Title: Melie Handbag Monogram Canvas
Standard Price: $1,460
Image URL: https://shop.rebag.com/cdn/shop/files/280868-171_20Louis_20Vuitton_20Melie_20Handbag_20Monogram_20Canvas_2D_0002_508x508.jpg?v=1719761493
Size and Fit: 12"W x 13"H x 5"D
Exterior Color: Brow

Brand: Goyard
Title: Saint Louis Tote Coated Canvas PM
Standard Price: $1,635
Image URL: https://shop.rebag.com/cdn/shop/files/284278-1_20Goyard_20Saint_20Louis_20Tote_20Coated_20Canvas_20PM_2D_0002_508x508.jpg?v=1719250604
Size and Fit: 13"W x 11"H x 6"D
Exterior Color: White
Interior Color: Neutral
Exterior Material: Canvas Coated
Interior Material: Fabric
Hardware Color: Silver
Accessories: Pochette
Store Location: Sunrise - Sawgrass Mills
CLAIR CODE: HB.GY.SL.COAC.PM
Item No: 284278/1
Brand Code: MAE020162
_____________________________________________________________________________________________
Product URL: https://shop.rebag.com/products/handbags-louis-vuitton-neverfull-nm-tote-monogram-canvas-gm2628181?groupBy=productClicked
Brand: Louis Vuitton
Title: Neverfull NM Tote Monogram Canvas GM
Standard Price: $1,200
Image URL: https://shop.rebag.com/cdn/shop/products/262818-1_20Louis_20Vuitton_20Neverfull_20NM_20Tote_20Monogram_20Canvas_20GM_2D_0002_508x508.jpg?v=1709579756
Size a

Brand: Gucci
Title: Sukey Hobo GG Canvas Medium
Standard Price: $650
Image URL: https://shop.rebag.com/cdn/shop/files/276634-1_20Gucci_20Sukey_20Hobo_20GG_20Canvas_20Medium_2D_0002_508x508.jpg?v=1716560753
Size and Fit: 14.5"W x 11.5"H x 3.5"D
Exterior Color: Brown
Interior Color: Brown
Exterior Material: Canvas Plain
Interior Material: Fabric
Hardware Color: Gold Tone
Accessories: Charms
Store Location: Online Only
Estimated Retail: $950
CLAIR CODE: HB.GC.SUH.GCV.ME
Item No: 276634/1
Brand Code: 362726 204991 F016236696
_____________________________________________________________________________________________
Product URL: https://shop.rebag.com/products/handbags-louis-vuitton-graceful-handbag-damier-pm2899171?groupBy=productClicked
Brand: Louis Vuitton
Title: Graceful Handbag Damier PM
Standard Price: $1,870
Image URL: https://shop.rebag.com/cdn/shop/files/289917-1_20Louis_20Vuitton_20Graceful_20Handbag_20Damier_20MM_2D_0002_508x508.jpg?v=1723487084
Size and Fit: 11.5"W x 10.5"H x 

In [1]:
import undetected_chromedriver as uc
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import pandas as pd
import time
from bs4 import BeautifulSoup


class RebagScraper:
    def __init__(self,brand_name):
        self.driver = uc.Chrome()
        self.driver.maximize_window()
        self.base_url = f'https://shop.rebag.com/search?q={brand_name}'
        self.page = 1
        self.brand_name = brand_name
        self.data = []
    def get_product_details(self, product_url):
        try:
            self.driver.get(product_url)
            WebDriverWait(self.driver, 10).until(EC.presence_of_element_located((By.CLASS_NAME, 'pdp__description')))
            soup_product = BeautifulSoup(self.driver.page_source, 'html.parser')

            img_tag = soup_product.find('div', class_='pdp__desktop-image')
            img_src = 'Image src attribute not found'
            if img_tag:
                img = img_tag.find('img')
                if img and img.has_attr('src'):
                    img_src = img['src']
                    if img_src.startswith('//'):
                        img_src = 'https:' + img_src

            designer_tag = soup_product.find('a', class_='pdp__designer')
            designer = designer_tag.text.strip() if designer_tag else 'Designer not found'

            title_tag = soup_product.find('h1', class_='pdp__header')
            title = 'Title not found'
            if title_tag:
                title_a_tag = title_tag.find('a', class_='pdp__title')
                if title_a_tag:
                    title = title_a_tag.text.strip()

            price_section = soup_product.find('div', class_='pdp__price')
            standard_price = 'Standard Price not found'
            if price_section:
                standard_price_tag = price_section.find('span', class_='rewards-plus-pdp__product-price-value')
                if standard_price_tag:
                    standard_price = standard_price_tag.text.strip()

            size_and_fit_div = soup_product.find('div', class_='pdp__section--size-and-fit')
            size_and_fit = 'Size and Fit not found'
            if size_and_fit_div:
                size_and_fit_tag = size_and_fit_div.find('div', class_='pdp__group-item')
                if size_and_fit_tag:
                    size_and_fit = size_and_fit_tag.text.strip()

            overview_div = soup_product.find('div', class_='pdp__section--overview')
            overview_details = {}
            if overview_div:
                groups = overview_div.find_all('div', class_='pdp__section-group')
                for group in groups:
                    title_tag = group.find('div', class_='pdp__group-title')
                    item_tag = group.find('div', class_='pdp__group-item')
                    if title_tag and item_tag:
                        section_title = title_tag.text.strip()
                        items = item_tag.find_all('div', class_='pdp__group-item--li')
                        if items:
                            item_list = [item.text.strip() for item in items]
                            overview_details[section_title] = ', '.join(item_list)
                        else:
                            overview_details[section_title] = item_tag.text.strip()

            additional_details_div = soup_product.find('div', class_='pdp__section--no-title')
            additional_details = {}
            if additional_details_div:
                groups = additional_details_div.find_all('div', class_='pdp__section-group')
                for group in groups:
                    title_tag = group.find('div', class_='pdp__group-title')
                    item_tag = group.find('div', class_='pdp__group-item')
                    if title_tag and item_tag:
                        section_title = title_tag.text.strip()
                        additional_details[section_title] = item_tag.get_text(strip=True)


            print(f"Product URL: {product_url}")
            print(f"Brand: {designer}")
            print(f"Title: {title}")
            print(f"Standard Price: {standard_price}")
            print(f"Image URL: {img_src}")
            print(f"Size and Fit: {size_and_fit}")

            for key, value in overview_details.items():
                print(f"{key}: {value}")

            for key, value in additional_details.items():
                print(f"{key}: {value}")

            print("_____________________________________________________________________________________________")

            product_data = {
                'Product URL': product_url,
                'Brand': designer,
                'Title': title,
                'Standard Price': standard_price,
                'Image URL': img_src,
                'Size and Fit': size_and_fit,
                
            }
            if overview_details:
                product_data.update(overview_details)
            if additional_details:
                product_data.update(additional_details)

            return product_data
        except:
            return {}

    def scrape_page(self, page_url):
        try:
            self.driver.get(page_url)
            print(f"Scraping page: {page_url}")

            WebDriverWait(self.driver, 5).until(
                EC.presence_of_element_located((By.CLASS_NAME, 'plp__product'))
            )
            soup = BeautifulSoup(self.driver.page_source, 'html.parser')
            products = soup.find_all('div', class_='plp__product')

            if not products:
                print(f"No products found on page: {page_url}")
                return False

            for product in products:
                try:
                    product_url_tag = product.find('a', class_='plp__card')
                    if product_url_tag:
                        product_url = product_url_tag.get('href')
                        if not product_url.startswith('http'):
                            product_url = 'https://shop.rebag.com' + product_url
                        product_data = self.get_product_details(product_url)
                        if product_data:
                            self.data.append(product_data)
                except:
                    continue

            return True
        except:
            return False
        

    def get_next_page(self,page_url):
        try:
            self.driver.get(page_url)
            next_button = WebDriverWait(self.driver, 10).until(
                EC.element_to_be_clickable((By.XPATH, '//ul[@class="rbg-pagination__ul"]//li[@class="next"]'))
            )
            if 'disabled' in next_button.get_attribute('class'):
                print("No more pages available, stopping.")
                return False
            else:
                next_button.click()
                time.sleep(3) 
                return True
        except:
            return False
        
  

    def run(self):
        if not self.scrape_page(self.base_url):
            print(f"Brand '{self.brand_name}' is not available or cannot be found.")
            self.driver.quit()
            return       
 
        while True:
            page_url = f'{self.base_url}&page={self.page}'
            print("page ",self.page)
            if not self.scrape_page(page_url):
                break
            if not self.get_next_page(page_url):
                break
            self.page += 1
        
        df = pd.DataFrame(self.data)
        df.to_csv('rebag_products.csv', index=False)
        print("Data saved to rebag_products.csv")



if __name__ == "__main__":
    brand_name = input("enter brand name: ").strip()
    scraper = RebagScraper(brand_name)
    scraper.run()

enter brand name: herms
Scraping page: https://shop.rebag.com/search?q=herms
Product URL: https://shop.rebag.com/products/accessories-hermes-hermy-horse-stuffed-animal-plush-1282472?groupBy=productClicked
Brand: Hermes
Title: Hermy Horse Stuffed Animal Plush
Standard Price: $825
Image URL: https://shop.rebag.com/cdn/shop/products/128247-2_20Hermes_20Hermy_20Horse_20Stuffed_20Animal_20Plush_2D_0002_508x508.jpg?v=1645119840
Size and Fit: 7.5"W x 8"H x 8"D
Exterior Color: Blue
Exterior Material: Plush
Accessories: Box
Store Location: Online Only
CLAIR CODE: HI.HM.HERM.PLUS
Item No: 128247/2
_____________________________________________________________________________________________
Product URL: https://shop.rebag.com/products/accessories-hermes-hermy-horse-stuffed-animal-plush-1373881?groupBy=productClicked
Brand: Hermes
Title: Hermy Horse Stuffed Animal Plush
Standard Price: $185
Image URL: https://shop.rebag.com/cdn/shop/products/137388-1_20Hermes_20Hermy_20Horse_20Stuffed_20Animal_20P

Product URL: https://shop.rebag.com/products/accessories-hermes-hermy-horse-stuffed-animal-plush-1342431?groupBy=productClicked
Brand: Hermes
Title: Hermy Horse Stuffed Animal Plush
Standard Price: $170
Image URL: https://shop.rebag.com/cdn/shop/products/134243-1_20Hermes_20Hermy_20Horse_20Stuffed_20Animal_20Plush_2D_0002_508x508.jpg?v=1647890605
Size and Fit: 7.5"W x 8.5"H x 8"D
Exterior Color: Blue
Exterior Material: Plush
Accessories: Box
Store Location: Online Only
CLAIR CODE: HI.HM.HERM.PLUS
Item No: 134243/1
_____________________________________________________________________________________________
Product URL: https://shop.rebag.com/products/accessories-hermes-hermy-horse-stuffed-animal-plush-1355583?groupBy=productClicked
Brand: Hermes
Title: Hermy Horse Stuffed Animal Plush
Standard Price: $230
Image URL: https://shop.rebag.com/cdn/shop/products/135558-3_20Hermes_20Hermy_20Horse_20Stuffed_20Animal_20Plush_2D_0002_508x508.jpg?v=1648487747
Size and Fit: 8.5"W x 8.5"H x 7"D
Ext

Product URL: https://shop.rebag.com/products/accessories-hermes-hermy-horse-stuffed-animal-plush-1342543?groupBy=productClicked
Brand: Hermes
Title: Hermy Horse Stuffed Animal Plush
Standard Price: $255
Image URL: https://shop.rebag.com/cdn/shop/products/134254-3_20Hermes_20Hermy_20Horse_20Stuffed_20Animal_20Plush_2D_0002_508x508.jpg?v=1647880739
Size and Fit: 7.5"W x 8.5"H x 8"D
Exterior Color: Pink
Exterior Material: Plush
Accessories: No accessories
Store Location: Online Only
CLAIR CODE: HI.HM.HERM.PLUS
Item No: 134254/3
_____________________________________________________________________________________________
Product URL: https://shop.rebag.com/products/accessories-hermes-hermy-horse-stuffed-animal-plush-1342394?groupBy=productClicked
Brand: Hermes
Title: Hermy Horse Stuffed Animal Plush
Standard Price: $170
Image URL: https://shop.rebag.com/cdn/shop/products/134239-4_20Hermes_20Hermy_20Horse_20Stuffed_20Animal_20Plush_2D_0002_508x508.jpg?v=1647883381
Size and Fit: 7.5"W x 8.5"